In [1]:
import pandas as pd
import numpy as np
import glob
import math 
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.io as pio

In [2]:
def fit_model(flow_data: pd.Series, doy, window=None):
    if window is not None:
        doy_min = (doy - window) % 365
        doy_max = (doy + window) % 365
        if doy_min > doy_max:
            day_list = list(range(doy_min, 366)) + list(range(1, doy_max+1))
        else:
            day_list = list(range(doy_min, doy_max + 1))
        flow_data_final = flow_data[flow_data.index.dayofyear.isin(day_list)]
    else:
        print(" Only one day of year is considered")
        # flow_data_final = flow_data[flow_data.index.dayofyear == doy]
        # params = genextreme.fit(flow_data_final, floc=0) # CHECK THIS
        # params = list(params)
    # return params, flow_data_final
    return flow_data_final

In [4]:

def get_max_min_percentiles():
    # Creating a function max and min from percentiles values
    file = "/Users/mugisha/Desktop/clone/Drought_Pakistan/riverflow_pakistan/flows/indus_at_tarbela.csv"
    flow_data = pd.read_csv(file, index_col=0, parse_dates=True)
    name_of_river = file.split("/")[-1].split(".")[0]

    # Define data storages: 
    list_of_values = []
    out_put_list = []

    for i in range(1, 366):
        out_put = fit_model(flow_data, i, window=10)
        out_put_list.append(out_put)

    for dataframe in out_put_list:
        # change values into a list
        values =dataframe.iloc[:, 0].values.tolist()
        clean_list = [x for x in values if not math.isnan(x)]
        list_of_values.append(clean_list)
 
    # loop through the percentages and compute dataframes
    dfs = {}
    percentages = [0.01, 0.1, 0.25, 0.75, 0.9, 0.99]
    for perc in percentages:
        df = pd.DataFrame(columns=[f'{perc*100}_%_min', f'{perc*100}_%_max'])
        for i, lst in enumerate(list_of_values):
            lower_bound = np.percentile(lst, perc*100 - 1) if perc != 0.01 else -np.inf
            upper_bound = np.percentile(lst, perc*100)
            values = [x for x in lst if lower_bound < x <= upper_bound]
            if values:
                df.loc[f'list{i+1}'] = [min(values), max(values)]
        dfs[perc] = df.reset_index(drop=True)

    return dfs

get_max_min_percentiles()[0.99]

,99.0_%_min,99.0_%_max
0,23062.617982,24970.318076
1,22805.173819,24212.464614
2,22224.953298,24000.022962
3,22200.021240,23834.950471
4,22000.021049,23599.983408
...,...,...
360,23704.992375,24970.318076
361,23100.022101,24812.461367
362,23062.617982,24652.132629
363,23000.022005,24652.132629


The Plot with ticks modified

In [5]:
# Get the data
percentile_dict = get_max_min_percentiles()
dfs = [percentile_dict[p] for p in [0.01, 0.1, 0.25, 0.75, 0.9, 0.99]]

# Set the index from 1 to 365
dfs = [df.set_index(pd.Index(range(1, 366))) for df in dfs]


# Create the traces
traces = []
fill_types = ['none'] + ['tonexty'] * 11
for i, df in enumerate(dfs):
    for j, col in enumerate(df.columns):
        fill = fill_types[j] if i == 0 else 'tonexty'
        traces.append(go.Scatter(
            x=df.index,
            y=df.iloc[:, j],
            name=col,
            fill=fill,
        ))

# Set the layout
layout = go.Layout(
    width=1200,
    height=600,
    title='Indus at Tarbela Dam Flow Percentiles (cfs)',
    xaxis=dict(title='', showticklabels=False, showgrid=False),
    yaxis=dict(title='Daily maximum and minimum discarge, in cubic feet per second',
               tickmode='array', showgrid=False,titlefont=dict(size=10)))

# Create the figure
fig = go.Figure(data=traces, layout=layout)

# Show the figure
pio.show(fig)